In [1]:
!pip install pandas python-chess stockfish

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 66.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for chess: filename=chess-1.11.2-py3-none-any.whl size=147776 sha256=764a0b9fc9240bb2a9eb7b2f4238aceb93ff3b926267ee3408daed829027707f
  Stored in directory: /root/.cache/pip/wheels/7a/79/8e/0d6e404db9f1e82af2e40b49161d6acab485d75dfb0470ac08
Successfully built chess


In [2]:
import os
!git clone https://github.com/official-stockfish/Stockfish.git
os.chdir('Stockfish/src')
!make -j build ARCH=x86-64 COMP=gcc

Cloning into 'Stockfish'...
remote: Enumerating objects: 38635, done.
remote: Counting objects: 100% (57/57), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 38635 (delta 32), reused 29 (delta 29), pack-reused 38578 (from 2)
Receiving objects: 100% (38635/38635), 17.84 MiB | 33.64 MiB/s, done.
Resolving deltas: 100% (29826/29826), done.
Successfully validated nn-1c0000000000.nnue
Successfully validated nn-37f18f62d772.nnue

Config:
debug: 'no'
sanitize: 'none'
optimize: 'yes'
arch: 'x86_64'
bits: '64'
kernel: 'Linux'
os: 'GNU/Linux'
prefetch: 'yes'
popcnt: 'no'
pext: 'no'
sse: 'yes'
mmx: 'no'
sse2: 'yes'
ssse3: 'no'
sse41: 'no'
avx2: 'no'
avxvnni: 'no'
avx512: 'no'
vnni256: 'no'
vnni512: 'no'
altivec: 'no'
vsx: 'no'
neon: 'no'
dotprod: 'no'
arm_version: '0'
lsx: 'no'
lasx: 'no'
target_windows: ''

Flags:
CXX: g++
CXXFLAGS:  -Wall -Wcast-qual -fno-exceptions -std=c++17  -pedantic -Wextra -Wshadow -Wmissing-declarations -m64 -DUSE_PTHREADS -DNDEBUG -O3 -funroll-loops

In [3]:
# Install python-chess if not already installed
!pip install python-chess

import chess
import chess.engine

In [4]:

# Set the correct engine path
engine_path = "/kaggle/working/Stockfish/src/stockfish"

# Initialize the Stockfish engine
engine = chess.engine.SimpleEngine.popen_uci(engine_path)

# Test the engine
# Create a new chess board
board = chess.Board()  # Start with the initial position

# Analyze the starting position
info = engine.analyse(board, chess.engine.Limit(time=0.1))
print(f"Score: {info['score']}")


# Clean up
engine.quit()

Score: PovScore(Cp(+38), WHITE)


In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import chess.pgn
import chess.engine
import io

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/chess-cheating-dataset/Games.csv


In [6]:
cd = pd.read_csv("/kaggle/input/chess-cheating-dataset/Games.csv")
cd.head()

,Liste cheat white,Liste cheat black,Game,Elo White,Elo Black,Score
0,00000000001111101010111010100110000001001111,0000000000111001000000001100100000000111000,1. d4 e6 2. c4 d5 3. Nc3 Be7 4. Nf3 Nf6 5. Bf4...,1001,-1,1/2-1/2
1,0000000000000010111101100000100111000111101111001,0000000000110111110111111111111111111011111111111,1. d4 c5 2. c4 cxd4 3. Qxd4 Nc6 4. Qd1 e6 5. N...,1001,-1,1/2-1/2
2,0000000000111001110111100111111110101011110111...,0000000000111111100001111111111111111111010101...,1. e4 e5 2. d4 exd4 3. Qxd4 Nc6 4. Qd3 b6 5. B...,1001,-1,1/2-1/2
3,000000000011111101111111110110101111111,00000000000001101001000011100111100110,1. e4 e5 2. Nf3 Nf6 3. Nc3 Bc5 4. Bc4 d6 5. O-...,1002,-1,1/2-1/2
4,0000000000001000111010111101001010011010011011...,0000000000000000010000001110011010100010001100...,1. d4 d5 2. c4 Nf6 3. Nc3 e6 4. Bg5 Be7 5. e3 ...,1002,-1,1/2-1/2


In [7]:
cd[['score_1', 'score_2']] = cd['Score'].str.split('-', expand=True)

cd['result'] = cd.apply(lambda row: 0 if row['score_1'] == '1' else (1 if row['score_2'] == '1' else 2), axis=1)

cd = cd.drop(columns=['score_1', 'score_2'])

cd

,Liste cheat white,Liste cheat black,Game,Elo White,Elo Black,Score,result
0,00000000001111101010111010100110000001001111,0000000000111001000000001100100000000111000,1. d4 e6 2. c4 d5 3. Nc3 Be7 4. Nf3 Nf6 5. Bf4...,1001,-1,1/2-1/2,2
1,0000000000000010111101100000100111000111101111001,0000000000110111110111111111111111111011111111111,1. d4 c5 2. c4 cxd4 3. Qxd4 Nc6 4. Qd1 e6 5. N...,1001,-1,1/2-1/2,2
2,0000000000111001110111100111111110101011110111...,0000000000111111100001111111111111111111010101...,1. e4 e5 2. d4 exd4 3. Qxd4 Nc6 4. Qd3 b6 5. B...,1001,-1,1/2-1/2,2
3,000000000011111101111111110110101111111,00000000000001101001000011100111100110,1. e4 e5 2. Nf3 Nf6 3. Nc3 Bc5 4. Bc4 d6 5. O-...,1002,-1,1/2-1/2,2
4,0000000000001000111010111101001010011010011011...,0000000000000000010000001110011010100010001100...,1. d4 d5 2. c4 Nf6 3. Nc3 e6 4. Bg5 Be7 5. e3 ...,1002,-1,1/2-1/2,2
...,...,...,...,...,...,...,...
48928,00000000000000000000000000000,00000000001001000000110010111,1. e4 e5 2. Qh5 Qe7 3. Bc4 Nf6 4. Qf3 d6 5. h3...,1841,1806,0-1,1
48929,0000000000000000000000000000000000000000,0000000000110011011110011111101110101001,1. c4 d5 2. c5 c6 3. b4 Bf5 4. d3 e6 5. Bf4 Nd...,1562,1561,0-1,1
48930,0000000000000000000000000000000000000000000000...,0000000000100110100100000100001000100100100110...,1. e4 e5 2. Bc4 Nf6 3. Nf3 d6 4. Ng5 Be6 5. Bx...,1476,1420,1-0,0
48931,0000000000000000000000000000000,000000000010000001000000010100,1. d4 Nf6 2. c4 g6 3. Nc3 e6 4. Nf3 Bg7 5. e4 ...,1745,1536,1-0,0


In [8]:
count = len(cd[(cd['Elo White'] < 1100) | (cd['Elo Black'] < 1100)])

print("Count of rows with 'feature' less than 1000:", count)

Count of rows with 'feature' less than 1000: 2897


In [9]:
cd_filtered = cd[(cd['Elo White'] < 1100) | (cd['Elo Black'] < 1100)]
cd_filtered

,Liste cheat white,Liste cheat black,Game,Elo White,Elo Black,Score,result
0,00000000001111101010111010100110000001001111,0000000000111001000000001100100000000111000,1. d4 e6 2. c4 d5 3. Nc3 Be7 4. Nf3 Nf6 5. Bf4...,1001,-1,1/2-1/2,2
1,0000000000000010111101100000100111000111101111001,0000000000110111110111111111111111111011111111111,1. d4 c5 2. c4 cxd4 3. Qxd4 Nc6 4. Qd1 e6 5. N...,1001,-1,1/2-1/2,2
2,0000000000111001110111100111111110101011110111...,0000000000111111100001111111111111111111010101...,1. e4 e5 2. d4 exd4 3. Qxd4 Nc6 4. Qd3 b6 5. B...,1001,-1,1/2-1/2,2
3,000000000011111101111111110110101111111,00000000000001101001000011100111100110,1. e4 e5 2. Nf3 Nf6 3. Nc3 Bc5 4. Bc4 d6 5. O-...,1002,-1,1/2-1/2,2
4,0000000000001000111010111101001010011010011011...,0000000000000000010000001110011010100010001100...,1. d4 d5 2. c4 Nf6 3. Nc3 e6 4. Bg5 Be7 5. e3 ...,1002,-1,1/2-1/2,2
...,...,...,...,...,...,...,...
48873,0000000000000000000000000000000000000000000000...,0000000000010011100010001010000010010000001000...,1. e4 c5 2. Nc3 e6 3. Nf3 Nc6 4. Be2 Qc7 5. O-...,-1,1778,1/2-1/2,2
48874,0000000000000000000000000000000000000000000000...,0000000000111111111111111111111111111111111111...,1. e4 a6 2. Nc3 b5 3. a4 e6 4. axb5 Bb7 5. bxa...,-1,1489,0-1,1
48886,0000000000000000000000000000000000000000000000...,0000000000011111111111111111111100011111101111...,1. d4 d5 2. c4 e6 3. Nc3 c6 4. e3 Nf6 5. Nf3 N...,-1,1954,1/2-1/2,2
48893,0000000000000000000000000000000000000000000000...,0000000000000101101111001011110001001000011111...,1. e4 e5 2. Nf3 Nf6 3. Nxe5 Nxe4 4. Qe2 d5 5. ...,-1,1839,1/2-1/2,2


In [10]:
pgn_data = cd_filtered['Game'].iloc[0]
# Treat the PGN data as a file-like object
pgn_io = io.StringIO(pgn_data)
# Parse the PGN data
game = chess.pgn.read_game(pgn_io)

In [11]:
# Initialize the chess engine (assuming Stockfish is installed and accessible)
engine = chess.engine.SimpleEngine.popen_uci(engine_path)


In [12]:
# Analyze each position in the game
board = game.board()
for move in game.mainline_moves():
    board.push(move)
    info = engine.analyse(board, chess.engine.Limit(time=0.1))
    print(f"Move: {move}, Score: {info['score']}")

Move: d2d4, Score: PovScore(Cp(-18), BLACK)
Move: e7e6, Score: PovScore(Cp(+39), WHITE)
Move: c2c4, Score: PovScore(Cp(-28), BLACK)
Move: d7d5, Score: PovScore(Cp(+25), WHITE)
Move: b1c3, Score: PovScore(Cp(-30), BLACK)
Move: f8e7, Score: PovScore(Cp(+30), WHITE)
Move: g1f3, Score: PovScore(Cp(-31), BLACK)
Move: g8f6, Score: PovScore(Cp(+29), WHITE)
Move: c1f4, Score: PovScore(Cp(-44), BLACK)
Move: e8g8, Score: PovScore(Cp(+35), WHITE)
Move: e2e3, Score: PovScore(Cp(-28), BLACK)
Move: a7a6, Score: PovScore(Cp(+40), WHITE)
Move: h2h3, Score: PovScore(Cp(+3), BLACK)
Move: d5c4, Score: PovScore(Cp(+9), WHITE)
Move: f1c4, Score: PovScore(Cp(-4), BLACK)
Move: b7b5, Score: PovScore(Cp(+25), WHITE)
Move: c4b3, Score: PovScore(Cp(+23), BLACK)
Move: c8b7, Score: PovScore(Cp(+2), WHITE)
Move: b3c2, Score: PovScore(Cp(+15), BLACK)
Move: b8d7, Score: PovScore(Cp(-19), WHITE)
Move: e1g1, Score: PovScore(Cp(+28), BLACK)
Move: a8c8, Score: PovScore(Cp(-23), WHITE)
Move: d1e2, Score: PovScore(Cp(+25),